<a href="https://colab.research.google.com/github/HS587885/MachineLearning/blob/main/Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install prophet
!pip install pystan
!pip install --upgrade setuptools
!pip install plotly

In [ ]:
import seaborn as sns
from prophet import Prophet

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from warnings import filterwarnings
filterwarnings("ignore")
import prophet



from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
filepath='/content/drive/Shareddrives/팔당댐/combined_data/filled_data(2012-2022).csv'

In [ ]:
# 전처리한 데이터 불러오기
df=pd.read_csv(filepath, index_col='ymdhm', parse_dates=['ymdhm'])
df

,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630,rf_10184100,rf_10184110,rf_10184140
ymdhm,,,,,,,,,,,,,,,,,
2012-05-01 00:00:00,24.800,555.00,219.07,24.93,555.00,445.0,310.7,469.05,300.2,0.0,290.0,729.80,275.3,540.18,0.0,0.0,0.0
2012-05-01 00:10:00,24.794,464.60,218.86,25.15,562.90,449.0,314.7,498.00,300.2,0.0,290.0,731.48,275.3,540.18,0.0,0.0,0.0
2012-05-01 00:20:00,24.789,478.10,218.69,25.31,576.40,451.0,313.7,490.68,301.2,0.0,290.0,726.42,275.3,540.18,0.0,0.0,0.0
2012-05-01 00:30:00,24.789,464.80,218.69,25.31,563.10,452.0,311.7,476.21,301.2,0.0,290.0,726.42,276.3,552.17,0.0,0.0,0.0
2012-05-01 00:40:00,24.789,478.10,218.69,25.31,576.40,450.0,311.7,476.21,301.2,0.0,291.0,707.17,277.3,564.29,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-18 23:10:00,25.040,259.23,212.86,31.14,259.23,510.0,0.0,319.84,0.0,0.0,0.0,-456.41,0.0,974.40,0.0,0.0,0.0
2022-07-18 23:20:00,25.040,260.46,212.86,31.14,260.46,492.0,0.0,314.01,0.0,0.0,0.0,-717.30,0.0,1006.88,0.0,0.0,0.0
2022-07-18 23:30:00,25.040,259.37,212.86,31.14,259.37,475.0,0.0,387.55,0.0,0.0,0.0,-843.37,0.0,1039.90,0.0,0.0,0.0


## train dataset 분리

In [ ]:
train_data=df.loc[:'2022-06-01',
                  ['swl', 'inf', 'sfw', 'ecpc', 'tototf', 'tide_level', 'fw_1018662', 'fw_1018680', 
                   'fw_1018683','fw_1019630', 'rf_10184100', 'rf_10184110', 'rf_10184140']]
train_label=df.loc[:'2022-06-01',['wl_1018662', 'wl_1018680', 'wl_1018683', 'wl_1019630']]


In [ ]:
y1 = train_label.iloc[:,0] #청담대교
y2 = train_label.iloc[:,1] #잠수교
y3 = train_label.iloc[:,2] #한강대교 
y4 = train_label.iloc[:,3] #행주대교


In [ ]:
# data=df.loc[:'2022-06-01',
#                   ['swl', 'inf', 'sfw', 'ecpc', 'tototf', 'tide_level', 'fw_1018662', 'fw_1018680', 
#                    'fw_1018683','fw_1019630', 'rf_10184100', 'rf_10184110', 'rf_10184140', 'wl_1018662']]
# data.reset_index()

In [ ]:
#data.info()

# PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler= StandardScaler()
X_scaled = scaler.fit_transform(data)

In [ ]:
X_scaled

array([[-1.92919647,  0.01078071, -0.05706133, ..., -0.10170421,
        -0.09792829, -0.23843154],
       [-1.97031534, -0.07099621, -0.07823439, ..., -0.10170421,
        -0.09792829, -0.18606695],
       [-2.00458106, -0.05878395, -0.09537449, ..., -0.10170421,
        -0.09792829, -0.1991581 ],
       ...,
       [ 0.33234135, -0.37797628, -0.35146771, ..., -0.10170421,
        -0.09792829, -4.30585126],
       [ 0.2638099 , -0.49127893, -0.38776439, ..., -0.10170421,
        -0.09792829, -4.30585126],
       [ 0.2638099 , -0.37799437, -0.38776439, ..., -0.10170421,
        -0.09792829, -4.30585126]])

In [ ]:
pca=PCA(n_components=2)
principalComponents = pca.fit_transform(X_scaled)
principalDf = pd.DataFrame(data=principalComponents, columns = ['ds','y'])
print(principalDf.head())
print(pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_))

         ds         y
0  0.113988 -1.060044
1  0.131321 -1.103442
2  0.143975 -1.139211
3  0.122317 -1.146052
4  0.129654 -1.143765
[0.44796604 0.18244666]
0.6304126971193093


In [ ]:
y1 = y1.reset_index()

In [ ]:
y1.columns = ['ds', 'y']

In [ ]:
y1

,ds,y
0,2012-05-01 00:00:00,310.7
1,2012-05-01 00:10:00,314.7
2,2012-05-01 00:20:00,313.7
3,2012-05-01 00:30:00,311.7
4,2012-05-01 00:40:00,311.7
...,...,...
269563,2022-06-01 23:10:00,0.0
269564,2022-06-01 23:20:00,0.0
269565,2022-06-01 23:30:00,0.0
269566,2022-06-01 23:40:00,0.0


In [ ]:
m = Prophet()
m.fit(y1)
future = m.make_future_dataframe(periods = 30)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpa_x63otv/uz506k8n.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpa_x63otv/glvptokz.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=3908', 'data', 'file=/tmp/tmpa_x63otv/uz506k8n.json', 'init=/tmp/tmpa_x63otv/glvptokz.json', 'output', 'file=/tmp/tmph0qa55ty/prophet_model-20220810081650.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
08:16:50 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
08:34:57 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [ ]:
forecast = m.predict(future)
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2012-05-01 00:00:00,229.414057,245.280167,409.248677,229.414057,229.414057,99.101287,99.101287,99.101287,8.734557,...,3.257493,3.257493,3.257493,87.109237,87.109237,87.109237,0.0,0.0,0.0,328.515344
1,2012-05-01 00:10:00,229.411322,243.943126,410.255861,229.411322,229.411322,98.652468,98.652468,98.652468,8.311099,...,3.282065,3.282065,3.282065,87.059304,87.059304,87.059304,0.0,0.0,0.0,328.063790
2,2012-05-01 00:20:00,229.408588,242.581817,412.893545,229.408588,229.408588,98.121692,98.121692,98.121692,7.805902,...,3.306297,3.306297,3.306297,87.009493,87.009493,87.009493,0.0,0.0,0.0,327.530279
3,2012-05-01 00:30:00,229.405853,246.566203,408.488737,229.405853,229.405853,97.518703,97.518703,97.518703,7.228716,...,3.330182,3.330182,3.330182,86.959804,86.959804,86.959804,0.0,0.0,0.0,326.924556
4,2012-05-01 00:40:00,229.403119,239.367025,409.485058,229.403119,229.403119,96.853636,96.853636,96.853636,6.589679,...,3.353718,3.353718,3.353718,86.910238,86.910238,86.910238,0.0,0.0,0.0,326.256755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269593,2022-06-27 23:50:00,155.963392,181.869012,353.400515,155.963392,155.963392,109.806963,109.806963,109.806963,9.067177,...,3.232585,3.232585,3.232585,97.507201,97.507201,97.507201,0.0,0.0,0.0,265.770355
269594,2022-06-28 23:50:00,155.840028,186.794897,353.323711,155.840028,155.840028,113.576533,113.576533,113.576533,9.067177,...,2.371974,2.371974,2.371974,102.137381,102.137381,102.137381,0.0,0.0,0.0,269.416561
269595,2022-06-29 23:50:00,155.716665,188.322282,353.083535,155.716665,155.716665,115.042689,115.042689,115.042689,9.067177,...,-0.974019,-0.974019,-0.974019,106.949531,106.949531,106.949531,0.0,0.0,0.0,270.759353
269596,2022-06-30 23:50:00,155.593301,192.431776,359.890544,155.593301,155.593301,121.838012,121.838012,121.838012,9.067177,...,0.897407,0.897407,0.897407,111.873427,111.873427,111.873427,0.0,0.0,0.0,277.431313


In [ ]:
sub_df = pd.read_csv('/content/drive/Shareddrives/팔당댐/competition_data/sample_submission.csv',index_col=0)
sub_df.head()

,wl_1018662,wl_1018680,wl_1018683,wl_1019630
ymdhm,,,,
2022-06-01 00:00,0,0,0,0
2022-06-01 00:10,0,0,0,0
2022-06-01 00:20,0,0,0,0
2022-06-01 00:30,0,0,0,0
2022-06-01 00:40,0,0,0,0


In [ ]:
sub_df.tail()

,wl_1018662,wl_1018680,wl_1018683,wl_1019630
ymdhm,,,,
2022-07-18 23:10,0,0,0,0
2022-07-18 23:20,0,0,0,0
2022-07-18 23:30,0,0,0,0
2022-07-18 23:40,0,0,0,0
2022-07-18 23:50,0,0,0,0


In [ ]:
len(sub_df['wl_1018662'])

6912